In [1]:
# temporarly add to path for testing purposes before package creation
import sys
sys.path.append("../../")

In [2]:
import numpy as np
from phdtools.ASEtools import asetools as at
from phdtools.ASEtools import atomstools

# ASE tools for handling trajs

In [3]:
!ls ../data/

traj_2.1_0-100-1.xyz


We will use this small trajectory of Li-ion electrolytes as a tutorial.

In [4]:
# init the project name.

project_dict = dict(
    projectName = "example_1_LiElectrolytes",
    trajPath = "../data/traj_2.1_0-100-1.xyz",
)

## Creation of the ASE _universe_

In [5]:
# init the Universe

aseU = at.Universe(**project_dict)

Gathering the Universe ...



Total atoms: 8402
Atom types: ['C' 'F' 'H' 'Li' 'O' 'P']

!!! Warning !!!
`rcutCorrection` not set, default values will be used!
rcutCorrection = {'C': 1.0, 'F': 1.0, 'H': 1.0, 'Li': 1.0, 'O': 1.0, 'P': 1.0}

0.2868s


In [6]:
# this property refers on how the molecules will be 
# automatically identified by functions of the pkg.

print(f"Default: {aseU.rcutCorrection}")

Default: {'C': 1.0, 'F': 1.0, 'H': 1.0, 'Li': 1.0, 'O': 1.0, 'P': 1.0}


In [7]:
# In this case we need to apply a correction, since the Li-ions and 
# the molecules in their SS will be identified as singular clusters.

rcut_dict = {'H': 1.0, 'C': 1.0, 'O': 1.0, 'Li': 0.1, 'P': 1.0, 'F': 1.0}

# The defaul obj can be changed by the function

aseU.set_rcut_correction(rcut_dict=rcut_dict)
print(f"Modified: {aseU.rcutCorrection}")

Updated rcutCorrection = {'H': 1.0, 'C': 1.0, 'O': 1.0, 'Li': 0.1, 'P': 1.0, 'F': 1.0}

Modified: {'H': 1.0, 'C': 1.0, 'O': 1.0, 'Li': 0.1, 'P': 1.0, 'F': 1.0}


In [8]:
# obtain the info about the molecules in the system
# !!! it is based on the rcut correction !!!

aseU.find_molecs()

# The same information can be accessed by the following command
# once they are identified

print(aseU.mol_dict)

Molecules found: {'mol1': 'C3H4O3', 'mol2': 'C4H8O3', 'mol3': 'F6P', 'mol4': 'Li'}

1.7085s
{'mol1': 'C3H4O3', 'mol2': 'C4H8O3', 'mol3': 'F6P', 'mol4': 'Li'}


In [9]:
# OPTIONAL
# Since we know how the system is composed we can also
# change the dictionary with the actual names of the mols

mol_names_list = ['EC', 'EMC', 'PF6', 'Li']

aseU.set_mol_names(mol_name_list=mol_names_list)

# the updated dictionary can be inspected as before

print(f"Updated: {aseU.mol_dict}")

Updated: {'EC': 'C3H4O3', 'EMC': 'C4H8O3', 'PF6': 'F6P', 'Li': 'Li'}
